In [1]:
library(terra)
library(biomod2)
# library(doParallel)

terra 1.7.29

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

biomod2 4.2-3 loaded.
 /!\ Since version 4.2 biomod2 relies on terra and may thus return SpatRaster that can easily be converted in RasterStack with `stack()`. We apologize for the trouble @('_')@



In [3]:
library(corrplot)

reefCSV  <- read.csv('env_var/photozoan.csv')
cor.res <- cor(reefCSV)
corrplot.mixed(cor.res, tl.cex = 0.7, tl.col = "black", addCoefasPercent = TRUE)

Warning message in file(file, "rt"):
“cannot open file 'env_var/photozoan_res1.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [27]:
# library(rmcfs)

In [2]:
library(terra)
library(biomod2)
library(doParallel)
registerDoParallel(cores=8)

res <- 'res3' # 1 degree resolution
reef <- 'photozoan'
reefName <- paste0(reef,'_',res)

# Read environmental dataset
files <- list.files(path=paste0('env_var/bio_',res,'_0Ma_foster'), pattern='.tif', full.names=TRUE )
nb_env <- length(files)
enviVar <- rast(files[1:nb_env])
crs(enviVar) <- '+proj=longlat +datum=WGS84 +no_defs'
enviVar <- rev(enviVar)
enviVar <- rev(enviVar)

# Read PA file
filename <- file.path(paste0('env_var/factoryReef3/',res,'_',reef,'.csv'))# 'env_var/factory_reef_0Ma', 'photozoan_large.csv')
reefFile  <- read.csv(filename)
dfReef <- as.numeric(reefFile[, reef])
reefXY <- reefFile[, c('lon','lat')]

reefBiomodData <- BIOMOD_FormatingData(resp.var = dfReef,
                                     expl.var = enviVar,
                                     resp.xy = reefXY,
                                     resp.name = reefName)

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel




-=-=-=-=-=-=-=-=-=-=-=-= photozoan_res3 Data Formating -=-=-=-=-=-=-=-=-=-=-=-=

      ! Response variable name was converted into photozoan.res3
      ! No data has been set aside for modeling evaluation
      ! No data has been set aside for modeling evaluation
      ! No data has been set aside for modeling evaluation
 ! Some NAs have been automatically removed from your data
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= Done -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [3]:
enviVar

class       : SpatRaster 
dimensions  : 360, 720, 8  (nrow, ncol, nlyr)
resolution  : 0.5, 0.5  (x, y)
extent      : -180, 180, -90, 90  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 +no_defs 
source(s)   : memory
names       :   bathy,         curr,   fstream,   saltMax,  saltMin,      sed, ... 
min values  : -9643.5,  0.008730656, -43.73863,  5.000838,  5.00007,    0.000, ... 
max values  :  2527.0, 53.702013237, 128.66075, 47.371456, 45.11129, 8884.758, ... 

In [2]:
plot(reefBiomodData);

ERROR: Error in plot(reefBiomodData): object 'reefBiomodData' not found


In [2]:
res <- 'res2'
myRespName <- 'photozoan'

In [4]:
# Read PA file
filename <- file.path(paste0('env_var/factoryReef/res4_',myRespName,'.csv'))# 'env_var/factory_reef_0Ma', 'photozoan_large.csv')
myRespF  <- read.csv(filename)
myResp <- as.numeric(myRespF[, myRespName])
myRespXY <- myRespF[, c('longitude','latitude')]

# Read environmental dataset
files <- list.files(path=paste0('env_var/bio_',res,'_0Ma_smooth'), pattern='.tif', full.names=TRUE )
nb_env <- length(files)
myExpl <- rast(files[1:nb_env])

#Define each environmental variable name
names(myExpl) <- c("z","cur","mix","sss","sst")

# Flip the data twice to ensure correct representation of PA points
crs(myExpl) <- '+proj=longlat +datum=WGS84 +no_defs'
myExpl <- rev(myExpl)
myExpl <- rev(myExpl)

# Format Data with true absences
myBiomodData <- BIOMOD_FormatingData(resp.var = myResp,
                                     expl.var = myExpl,
                                     resp.xy = myRespXY,
                                     resp.name = myRespName)

# plot(myBiomodData)

ERROR: Error in `[.data.frame`(myRespF, , c("longitude", "latitude")): undefined columns selected


In [ ]:
# Default modelling options
myBiomodOptions <- BIOMOD_ModelingOptions()

# Model single models
myBiomodModelOut <- BIOMOD_Modeling(bm.format = myBiomodData,
                                    bm.options = myBiomodOptions,
                                    modeling.id = 'MAXENT',
                                    CV.strategy = 'random',
                                    CV.nb.rep = 2,
                                    CV.perc = 0.8,
                                    var.import = 3,
                                    metric.eval = c('TSS','ROC'),
                                    seed.val = 123,
                                    nb.cpu = 8)

In [ ]:
myBiomodOptions

In [ ]:
# Model ensemble models
myBiomodEM <- BIOMOD_EnsembleModeling(bm.mod = myBiomodModelOut,
                                      models.chosen = 'all',
                                      em.by = 'all',
                                      em.algo = c('EMmean', 'EMcv', 'EMci', 'EMmedian', 'EMca', 'EMwmean'),
                                      metric.select = c('TSS'),
                                      metric.select.thresh = c(0.7),
                                      metric.eval = c('TSS', 'ROC'),
                                      var.import = 3,
                                      EMci.alpha = 0.05,
                                      EMwmean.decay = 'proportional')

In [ ]:
myBiomodModelOut

In [ ]:
# Print default modelling options
bm_DefaultModelingOptions()

# Create default modeling options
myBiomodOptions <- BIOMOD_ModelingOptions()
myBiomodOptions

In [ ]:
# Model single models
myBiomodModelOut <- BIOMOD_Modeling(bm.format = myBiomodData,
                                    bm.options = myBiomodOptions,
                                    modeling.id = 'AllModels',
                                    CV.strategy = 'random',
                                    CV.nb.rep = 2,
                                    CV.perc = 0.8,
                                    var.import = 3,
                                    metric.eval = c('TSS','ROC'),
                                    seed.val = 123,
                                    nb.cpu = 8)
myBiomodModelOut

# Get evaluation scores & variables importance
get_evaluations(myBiomodModelOut)
get_variables_importance(myBiomodModelOut)

# Represent evaluation scores & variables importance
bm_PlotEvalMean(bm.out = myBiomodModelOut)
bm_PlotEvalBoxplot(bm.out = myBiomodModelOut, group.by = c('algo', 'algo'))
bm_PlotEvalBoxplot(bm.out = myBiomodModelOut, group.by = c('algo', 'run'))
bm_PlotVarImpBoxplot(bm.out = myBiomodModelOut, group.by = c('expl.var', 'algo', 'algo'))
bm_PlotVarImpBoxplot(bm.out = myBiomodModelOut, group.by = c('expl.var', 'algo', 'run'))
bm_PlotVarImpBoxplot(bm.out = myBiomodModelOut, group.by = c('algo', 'expl.var', 'run'))

# Represent response curves
bm_PlotResponseCurves(bm.out = myBiomodModelOut, 
                      models.chosen = get_built_models(myBiomodModelOut)[c(1:3, 12:14)],
                      fixed.var = 'median')
bm_PlotResponseCurves(bm.out = myBiomodModelOut, 
                      models.chosen = get_built_models(myBiomodModelOut)[c(1:3, 12:14)],
                      fixed.var = 'min')
bm_PlotResponseCurves(bm.out = myBiomodModelOut, 
                      models.chosen = get_built_models(myBiomodModelOut)[3],
                      fixed.var = 'median',
                      do.bivariate = TRUE)

In [ ]:
# Model ensemble models
myBiomodEM <- BIOMOD_EnsembleModeling(bm.mod = myBiomodModelOut,
                                      models.chosen = 'all',
                                      em.by = 'all',
                                      em.algo = c('EMmean', 'EMcv', 'EMci', 'EMmedian', 'EMca', 'EMwmean'),
                                      metric.select = c('TSS'),
                                      metric.select.thresh = c(0.7),
                                      metric.eval = c('TSS', 'ROC'),
                                      var.import = 3,
                                      EMci.alpha = 0.05,
                                      EMwmean.decay = 'proportional')
myBiomodEM

# Get evaluation scores & variables importance
get_evaluations(myBiomodEM)
get_variables_importance(myBiomodEM)

# Represent evaluation scores & variables importance
bm_PlotEvalMean(bm.out = myBiomodEM, group.by = 'full.name')
bm_PlotEvalBoxplot(bm.out = myBiomodEM, group.by = c('full.name', 'full.name'))
bm_PlotVarImpBoxplot(bm.out = myBiomodEM, group.by = c('expl.var', 'full.name', 'full.name'))
bm_PlotVarImpBoxplot(bm.out = myBiomodEM, group.by = c('expl.var', 'algo', 'merged.by.run'))
bm_PlotVarImpBoxplot(bm.out = myBiomodEM, group.by = c('algo', 'expl.var', 'merged.by.run'))

# Represent response curves
bm_PlotResponseCurves(bm.out = myBiomodEM, 
                      models.chosen = get_built_models(myBiomodEM)[c(1, 6, 7)],
                      fixed.var = 'median')
bm_PlotResponseCurves(bm.out = myBiomodEM, 
                      models.chosen = get_built_models(myBiomodEM)[c(1, 6, 7)],
                      fixed.var = 'min')
bm_PlotResponseCurves(bm.out = myBiomodEM, 
                      models.chosen = get_built_models(myBiomodEM)[7],
                      fixed.var = 'median',
                      do.bivariate = TRUE)

In [ ]:
# Evaluate models with Boyce index and MPA
myBiomodPO <- BIOMOD_PresenceOnly(bm.mod = myBiomodModelOut,
                                  bm.em = myBiomodEM)
myBiomodPO

# Evaluate models with Boyce index and MPA (using background data)
myBiomodPO <- BIOMOD_PresenceOnly(bm.mod = myBiomodModelOut,
                                  bm.em = myBiomodEM, 
                                  bg.env = values(myExpl))
myBiomodPO

In [ ]:
# Project single models
myBiomodProj <- BIOMOD_Projection(bm.mod = myBiomodModelOut,
                                  proj.name = 'Current',
                                  new.env = myExpl,
                                  models.chosen = 'all',
                                  metric.binary = 'all',
                                  metric.filter = 'all',
                                  build.clamping.mask = TRUE)
myBiomodProj
plot(myBiomodProj)

In [ ]:
# Project ensemble models (from single projections)
myBiomodEMProj <- BIOMOD_EnsembleForecasting(bm.em = myBiomodEM, 
                                             bm.proj = myBiomodProj,
                                             models.chosen = 'all',
                                             metric.binary = 'all',
                                             metric.filter = 'all')
                                             
# Project ensemble models (building single projections)
myBiomodEMProj <- BIOMOD_EnsembleForecasting(bm.em = myBiomodEM,
                                             proj.name = 'CurrentEM',
                                             new.env = myExpl,
                                             models.chosen = 'all',
                                             metric.binary = 'all',
                                             metric.filter = 'all')
myBiomodEMProj
plot(myBiomodEMProj)

In [1]:
res <- 'res5' # 2 degree resolution
# res <- 'res6' # 3 degree resolution
reef <- 'photozoan'
reefName <- paste0(reef,'_',res)

In [2]:
reefName

[1] "photozoan_res5"